# Silent Eight — End-to-End Demo

This notebook can run the full project end-to-end.

Colab mode: you can upload **only this notebook** and it will download the full repo ZIP from Google Drive.

Pipeline:
1) Download repo (Colab only)
2) Install dependencies
3) Generate demo data + SQLite DB
4) Preprocess + build keys/LSH
5) Train model
6) Offline matching + evaluation
7) Start FastAPI + call `/resolve`


## 0) Setup

If `src/` and `requirements.txt` are not present in the current working directory, the next cell (on Colab) will download and unpack the repo from Google Drive.


In [1]:
from __future__ import annotations

import os
import shutil
import subprocess
import sys
from pathlib import Path
from typing import List, Optional


def running_in_colab() -> bool:
    try:
        import google.colab  # type: ignore
        return True
    except Exception:
        return False


def run_and_show(cmd: List[str], title: Optional[str] = None) -> None:
    if title:
        print("\n" + "=" * 80)
        print(title)
        print("$", " ".join(cmd))
        print("=" * 80)
    result = subprocess.run(cmd, capture_output=True, text=True)
    if result.stdout:
        print(result.stdout)
    if result.stderr:
        print("[stderr]\n" + result.stderr)
    result.check_returncode()


def find_repo_root(base: Path) -> Optional[Path]:
    for api_file in base.glob("**/src/api.py"):
        return api_file.parents[1]
    return None


def download_drive_zip(file_id: str, dest: Path) -> None:
    dest.parent.mkdir(parents=True, exist_ok=True)
    run_and_show([sys.executable, "-m", "pip", "install", "-q", "gdown"], title="Install gdown")
    import gdown  # type: ignore
    url = f"https://drive.google.com/uc?id={file_id}"
    print("Downloading:", url)
    gdown.download(url, str(dest), quiet=False, fuzzy=True)
    if not dest.exists() or dest.stat().st_size == 0:
        raise RuntimeError(
            "Download failed or produced an empty file. "
            "Check Drive sharing: 'Anyone with the link' and downloads allowed."
        )


DRIVE_FILE_ID = "12ZzXx72imAvG5wTrzwbJwDYji3Eyvzf-"  # <-- replace if you upload a new ZIP
ZIP_PATH = Path("/content/silent-eight-assignment.zip")


if running_in_colab():
    content_root = Path("/content")
    repo_root = find_repo_root(Path.cwd()) or find_repo_root(content_root)
    if repo_root is None:
        download_drive_zip(DRIVE_FILE_ID, ZIP_PATH)
        print("Unpacking ZIP to /content ...")
        shutil.unpack_archive(str(ZIP_PATH), str(content_root))
        repo_root = find_repo_root(content_root)
        if repo_root is None:
            raise RuntimeError("Could not locate repo root under /content after unpack.")
    os.chdir(repo_root)
else:
    cwd = Path.cwd()
    if cwd.name == "notebooks":
        os.chdir(cwd.parent)


project_root = Path.cwd()
print("Project root:", project_root)
print("Python:", sys.executable)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))


Project root: c:\repos\silent-eight-assignment
Python: c:\Users\kugor\anaconda3\envs\silent-eight\python.exe


## 1) Install dependencies

On Colab this installs `requirements.txt` from the downloaded repo.


In [2]:
if running_in_colab():
    run_and_show([
        sys.executable, "-m", "pip", "install", "-r", "requirements.txt"
    ], title="Install dependencies from requirements.txt")
else:
    print("Skipping install (not running on Colab).")


Skipping install (not running on Colab).


## 2) Generate demo data + SQLite database

This writes outputs under `data/`.


In [3]:
run_and_show([sys.executable, "-m", "src.generation"], title="Generate demo data + SQLite DB")

import pandas as pd

messy_csv = project_root / "data" / "messy_data.csv"
if messy_csv.exists():
    df = pd.read_csv(messy_csv)
    print("\nGenerated data preview (data/messy_data.csv):")
    display(df.head(5))
else:
    print("No data/messy_data.csv found")



Generate demo data + SQLite DB
$ c:\Users\kugor\anaconda3\envs\silent-eight\python.exe -m src.generation
Generating Ground Truth Entities (with 5% Doppelgangers)...
Generating Messy Observations...
------------------------------
Generated 2554 records.
Sample Data (First 10 rows):
  first_name last_name  ...           phone_number                         address
0       Alex   Duczmal  ...        +48 881 819 600           ulica Cyprysowa 08/38
1       Alex   Duczmal  ...        +48 881 819 600           ulica Cyprysowa 08/38
2       Alxe   Duczmal  ...            602 099 499           ulica Cyprysowa 08/38
3       Mark   Johnson  ...  001-218-519-6001x3389  79402 Peterson Drives Apt. 511
4       Mark   Johnson  ...                   None  79402 Peterson Drives Apt. 511
5     Marcus    Dowerg  ...       +49(0)1338908386                 Albersallee 565
6     Marcus    Dowerg  ...       +49(0)1338908386                 Albersallee 565
7         M.    Dowerg  ...       +49(0)1338908386   

,entity_id,first_name,last_name,dob,email,country,national_id,phone_number,address,city,record_id
0,db2588a4-1be3-457d-a08d-db9dc90e4620,Alex,Duczmal,1951-05-22,alex.duczmal@yahoo.com,PL,10320843322,+48 881 819 600,ulica Cyprysowa 08/38,Szczecinek,8b9db101-0eae-4f45-ad7e-14ce7f9a1009
1,db2588a4-1be3-457d-a08d-db9dc90e4620,Alex,Duczmal,22/05/1951,alex.duczmal@yahoo.com,PL,10320843322,+48 881 819 600,ulica Cyprysowa 08/38,Szczecinek,1369b2d5-8160-414d-a8b2-25fd87b8493a
2,db2588a4-1be3-457d-a08d-db9dc90e4620,Alxe,Duczmal,1951-05-22,alex.duczmal@yahoo.com,PL,10320843322,602 099 499,ulica Cyprysowa 08/38,Szczecinek,b684e36b-c7f1-40c3-ad69-afb2492df630
3,160a4ed6-41f6-4469-9275-c0b974f3c91b,Mark,Johnson,1993-09-26,mark.johnson@gmail.com,US,251-29-2287,001-218-519-6001x3389,79402 Peterson Drives Apt. 511,Davisstad,d439cde1-2115-489e-b02f-5cfca8cf3095
4,160a4ed6-41f6-4469-9275-c0b974f3c91b,Mark,Johnson,1993-09-26,NaN,US,251-29-2287,NaN,79402 Peterson Drives Apt. 511,Davisstad,773ffaad-edff-40b7-99e8-df7b2b64ecc2


## 3) Preprocess + build LSH artifacts

Creates normalized columns, blocking keys, DB indexes, and (optionally) LSH artifacts under `models/`.


In [4]:
run_and_show([sys.executable, "-m", "src.preprocessing"], title="Preprocess + build blocking keys/LSH")

import sqlite3
import pandas as pd

db_path = project_root / "data" / "clients.db"
if db_path.exists():
    with sqlite3.connect(db_path) as conn:
        try:
            df = pd.read_sql_query("SELECT * FROM clients_processed LIMIT 5", conn)
            print("\nProcessed table preview (clients_processed):")
            display(df)
        except Exception as e:
            print("Could not read clients_processed preview:", e)
else:
    print("No data/clients.db found after preprocessing.")



Preprocess + build blocking keys/LSH
$ c:\Users\kugor\anaconda3\envs\silent-eight\python.exe -m src.preprocessing
Loading raw data...
Normalizing and generating keys...
  > Generating MinHash signatures...
  > Generating ID keys...
  > Generating Phone keys...
  > Generating Email keys...
  > Generating Initial+DOB keys...

--- Date Parsing Check ---
          dob norm_dob_year
0  1951-05-22          1951
1  22/05/1951          1951
2  1951-05-22          1951
3  1993-09-26          1993
4  1993-09-26          1993
5  1960-01-06          1960
6  1960-01-06          1960
7  1960-01-06          1960
8  1975-11-13          1975
9  1969-06-10          1969

--- Blocking Keys Check ---
        bk_nid  ...               bk_email
0  10320843322  ...  alexduczmal@yahoo.com
1  10320843322  ...  alexduczmal@yahoo.com
2  10320843322  ...  alexduczmal@yahoo.com
3    251292287  ...  markjohnson@gmail.com
4    251292287  ...                   None

[5 rows x 4 columns]

--- Blocking Key Coverage --

,entity_id,first_name,last_name,dob,email,country,national_id,phone_number,address,city,...,norm_address,norm_city,norm_dob,norm_dob_year,bk_minhash,bk_nid,bk_phone,bk_email,bk_initial_dob,key_count
0,db2588a4-1be3-457d-a08d-db9dc90e4620,Alex,Duczmal,1951-05-22,alex.duczmal@yahoo.com,PL,10320843322,+48 881 819 600,ulica Cyprysowa 08/38,Szczecinek,...,ulica cyprysowa 0838,szczecinek,1951-05-22,1951,"[86265554, 883542472, 315088925, 71200896, 117...",10320843322,819600,alexduczmal@yahoo.com,a|duczmal|1951-05-22,4
1,db2588a4-1be3-457d-a08d-db9dc90e4620,Alex,Duczmal,22/05/1951,alex.duczmal@yahoo.com,PL,10320843322,+48 881 819 600,ulica Cyprysowa 08/38,Szczecinek,...,ulica cyprysowa 0838,szczecinek,1951-05-22,1951,"[86265554, 883542472, 315088925, 71200896, 117...",10320843322,819600,alexduczmal@yahoo.com,a|duczmal|1951-05-22,4
2,db2588a4-1be3-457d-a08d-db9dc90e4620,Alxe,Duczmal,1951-05-22,alex.duczmal@yahoo.com,PL,10320843322,602 099 499,ulica Cyprysowa 08/38,Szczecinek,...,ulica cyprysowa 0838,szczecinek,1951-05-22,1951,"[86265554, 194400707, 137792012, 1327957820, 9...",10320843322,099499,alexduczmal@yahoo.com,a|duczmal|1951-05-22,4
3,160a4ed6-41f6-4469-9275-c0b974f3c91b,Mark,Johnson,1993-09-26,mark.johnson@gmail.com,US,251-29-2287,001-218-519-6001x3389,79402 Peterson Drives Apt. 511,Davisstad,...,79402 peterson drives apartment 511,davisstad,1993-09-26,1993,"[234886539, 180801429, 387971711, 747788293, 3...",251292287,013389,markjohnson@gmail.com,m|johnson|1993-09-26,4
4,160a4ed6-41f6-4469-9275-c0b974f3c91b,Mark,Johnson,1993-09-26,None,US,251-29-2287,None,79402 Peterson Drives Apt. 511,Davisstad,...,79402 peterson drives apartment 511,davisstad,1993-09-26,1993,"[234886539, 180801429, 387971711, 747788293, 3...",251292287,None,None,m|johnson|1993-09-26,2


## 4) Train the model

Trains the classifier and writes the model artifact under `models/`.


In [5]:
run_and_show([sys.executable, "-m", "src.train_model"], title="Train model")

model_path = project_root / "models" / "entity_resolution_model.pkl"
print("\nModel exists:", model_path.exists(), "->", model_path)



Train model
$ c:\Users\kugor\anaconda3\envs\silent-eight\python.exe -m src.train_model
Connecting to database...
Loading processed data...
Loading ground truth...
Generating candidates via LSH & Blocking Keys...
  > Loading LSH Index from disk (Fast)...
  > Loaded LSH Index with 2554 records.
  > Querying LSH Index...
  > Found 1507 pairs via LSH.
  > Querying Exact Blocking Keys (Optimized with UNION)...
  > Found 2804 pairs via Exact Keys.
Total Unique Candidate Pairs: 3044

--- Blocking Performance Report ---
Total Records: 2,554
Candidate Pairs: 3,044
Reduction Ratio: 99.90663095%
Pairs per Record: 1.19

[Block Size Analysis - Top 3 Largest Blocks per Key]
Key: bk_nid
  '019475061': 4 records
  '027371999': 4 records
  '04240376Y753': 4 records
Key: bk_phone
  '333945': 10 records
  '800384': 8 records
  '606474': 7 records
Key: bk_email
  'sheliajohnson@outlook.com': 5 records
  'aaronkelly@example.net': 4 records
  'adacitko@hotmail.com': 4 records
------------------------------

## 5) Offline matching + evaluation

Runs candidate generation + scoring + clustering and outputs evaluation CSVs under `data/`.


In [6]:
run_and_show([sys.executable, "-m", "src.matching"], title="Offline matching + evaluation")

import pandas as pd

data_dir = project_root / "data"
csvs = sorted(data_dir.glob("*.csv")) if data_dir.exists() else []
print("\nData CSV outputs:")
for p in csvs:
    print("-", p.name)

preview = data_dir / "manual_review_cases.csv"
if preview.exists():
    df = pd.read_csv(preview)
    print("\nPreview of manual_review_cases.csv:")
    display(df.head(5))



Offline matching + evaluation
$ c:\Users\kugor\anaconda3\envs\silent-eight\python.exe -m src.matching
Loading processed data for lookup...
Generating candidates via LSH & Blocking Keys...
  > Loading LSH Index from disk (Fast)...
  > Loaded LSH Index with 2554 records.
  > Querying LSH Index...
  > Found 1507 pairs via LSH.
  > Querying Exact Blocking Keys (Optimized with UNION)...
  > Found 2804 pairs via Exact Keys.
Total Unique Candidate Pairs: 3044

--- Blocking Performance Report ---
Total Records: 2,554
Candidate Pairs: 3,044
Reduction Ratio: 99.90663095%
Pairs per Record: 1.19

[Block Size Analysis - Top 3 Largest Blocks per Key]
Key: bk_nid
  '019475061': 4 records
  '027371999': 4 records
  '04240376Y753': 4 records
Key: bk_phone
  '333945': 10 records
  '800384': 8 records
  '606474': 7 records
Key: bk_email
  'sheliajohnson@outlook.com': 5 records
  'aaronkelly@example.net': 4 records
  'adacitko@hotmail.com': 4 records
------------------------------

Calculating comparison

,id_a,id_b,nid_score,nid_both_present,email_score,phone_match,first_name_score,last_name_score,addr_score,dob_match,dob_both_present,year_match,ml_prob,match_type,confidence_score,explanation,is_match
0,82db8797-02e4-47e8-8e28-0f27cedf9155,8b1cf028-1580-4ad7-96da-4271d8a8df8b,0.222222,1,0.928485,0,1.000000,1.000000,0.166667,0,1,0,0.008697,review,0.50,Strong Name Match Only,0
1,30ca0ccd-99dc-4c38-a522-65bf5d950b36,cec346fe-3c86-4000-9067-ccc214a02b0e,0.111111,1,0.919908,0,1.000000,1.000000,0.190476,0,1,0,0.013526,review,0.50,Strong Name Match Only,0
2,30ca0ccd-99dc-4c38-a522-65bf5d950b36,660359ca-586f-4e9d-82e8-644a1bc0f231,0.111111,1,0.919908,0,1.000000,1.000000,0.190476,0,1,0,0.013526,review,0.50,Strong Name Match Only,0
3,0974d005-dbdf-4d81-847d-8f1d6763d03a,0a950df7-d61d-47ad-a770-900cccd62b4b,0.111111,1,1.000000,0,0.555556,0.980952,1.000000,0,0,0,0.488424,review,0.55,Moderate ML Probability (0.49); Exact Email Ma...,0
4,1366afd3-400c-4cef-8639-52ec2796f22e,fe325654-f971-4cbf-bc5e-1a12db55cb9a,0.083333,1,1.000000,0,0.000000,0.555556,0.187500,0,1,0,0.024081,review,0.55,Exact Email Match Only,0


## 6) Start the FastAPI service

Starts Uvicorn in the background.

In Colab, a new window will open via the Colab port proxy.


In [8]:
import time

db_path = project_root / "data" / "clients.db"
model_path = project_root / "models" / "entity_resolution_model.pkl"
lsh_index_path = project_root / "models" / "lsh_index.pkl"

env = os.environ.copy()
env["ER_DB_PATH"] = str(db_path)
env["ER_MODEL_PATH"] = str(model_path)
env["ER_LSH_INDEX_PATH"] = str(lsh_index_path)

host = "0.0.0.0" if running_in_colab() else "127.0.0.1"
proc = subprocess.Popen(
    [sys.executable, "-m", "uvicorn", "src.api:app", "--host", host, "--port", "8000"],
    env=env,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
 )

time.sleep(1.5)
print("Server started (pid):", proc.pid)

if running_in_colab():
    from google.colab import output  # type: ignore
    output.serve_kernel_port_as_iframe(8000, path="docs")
else:
    print("Docs:    http://127.0.0.1:8000/docs")
    print("Health:  http://127.0.0.1:8000/health")
    print("Metrics: http://127.0.0.1:8000/metrics")


Server started (pid): 17688
Docs:    http://127.0.0.1:8000/docs
Health:  http://127.0.0.1:8000/health
Metrics: http://127.0.0.1:8000/metrics


## 7) Call the API (`/health`, then `/resolve`)

If the request fails, re-run the previous cell and wait ~1–2 seconds.


In [9]:
import requests

base_url = "http://127.0.0.1:8000"

if running_in_colab():
    from google.colab import output  # type: ignore
    try:
        proxy_url = output.eval_js("google.colab.kernel.proxyPort(8000)").rstrip("/")
        base_url = proxy_url
        print("Using Colab proxy base_url:", base_url)
        print("Swagger:", base_url + "/docs")
    except Exception as e:
        print("Could not get Colab proxy URL; using localhost.", e)

print("GET /health")
print(requests.get(f"{base_url}/health", timeout=10).json())

payload = {
    "first_name": "John",
    "last_name": "Doe",
    "dob": "1980-01-01",
    "national_id": "A1234567",
    "email": "john.doe@example.com",
    "phone_number": "+1 (555) 010-0200",
}

print("POST /resolve")
resp = requests.post(f"{base_url}/resolve", json=payload, timeout=30)
resp.raise_for_status()
result = resp.json()
print("Status:", result.get("status"))
print("Candidates checked:", result.get("candidates_checked"))
print("Processing time (ms):", result.get("processing_time_ms"))
print("Best match:", result.get("best_match"))


ModuleNotFoundError: No module named 'requests'

## 8) Stop the server

Run when you're done.


In [ ]:
if "proc" in globals() and proc and proc.poll() is None:
    proc.terminate()
    try:
        proc.wait(timeout=5)
    except Exception:
        proc.kill()
    print("Server stopped.")
else:
    print("Server is not running.")
